In [ ]:
from youtubesearchpython import Hashtag, Comments
from tqdm import tqdm
import time
import pandas as pd
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
import utils
from datetime import datetime

In [ ]:
hashtag = Hashtag('marvel', limit = 60)

In [ ]:
data = []
df = pd.DataFrame()
for res in tqdm(hashtag.result()['result']):
    vid_id = res.get('id')
    data_dict = {}
    data_dict['id'] = vid_id
    data_dict['channel_id'] = res.get('channel').get('id')
    data_dict['text'] = res.get('title')
    data_dict['published_time'] = res.get('publishedTime')
    data_dict['duration'] = res.get('duration')
    data_dict['viewCount'] = res.get('viewCount').get('text')
    data.append(data_dict)
    try:
        com = Comments(vid_id)
    except:
        continue
    try:
        while com.hasMoreComments:
            com.getNextComments()
        for comment in com.comments['result']:
            data_dict = {}
            data_dict['id'] = comment.get('id')
            data_dict['video_id'] = vid_id
            data_dict['channel_id'] = comment.get('author').get('id')
            data_dict['text'] = comment.get('content')
            data_dict['published_time'] = comment.get('publishedTime')
            data_dict['likes'] = comment.get('votes').get('simpleText')
            data_dict['replies'] = comment.get('replyCount')
            data.append(data_dict)
    except Exception as e:
        print(f'No comments: {e}')
    time.sleep(10)
    df = pd.concat((df, pd.DataFrame(data)), axis=0, ignore_index=True).drop_duplicates()
    if df.shape[0] > 700000:
        break

In [10]:
df

,id,channel_id,text,published_time,duration,viewCount,video_id,likes,replies
0,116Vptosbg8,UCKy1dAqELo0zrOtPkf0eTMw,Marvel Snap - Official Cinematic Reveal Traile...,2 days ago,7:00,"521,446 views",NaN,NaN,NaN
1,UgxktLMG7OwYTr07GNx4AaABAg,UChP4bp6UDcqVDMiU7Ft1khg,They might as well have given her the elastici...,None,NaN,NaN,116Vptosbg8,19,3.0
2,UgxXGzflxinrpVapTmV4AaABAg,UCu3Wa2NDfLnxinXJO5QvXSg,Stunning and brave,None,NaN,NaN,116Vptosbg8,10,NaN
3,Ugw0lCUW_y3C5p0eIX54AaABAg,UCXtCLYV1HiNBSp3qIfclMBA,"HOHOHOHO...""If you save one life, you save the...",None,NaN,NaN,116Vptosbg8,5,NaN
4,UgysXnN5ZasAySKT_QB4AaABAg,UCx7DSYaFjXUvpYcX5CLgsPQ,I’m actually glad they did that too her powers...,None,NaN,NaN,116Vptosbg8,4,5.0
...,...,...,...,...,...,...,...,...,...
2321178,Ugxxo6Jm6M9Tiuegr0R4AaABAg,UCKlRBvdB9sLLkvt6B3r4wcw,Op avengers,None,NaN,NaN,b-ZyMZo4Bew,None,NaN
2321179,Ugw1eDzisVJyd-MMpaV4AaABAg,UCUw0-uGneGz4Jv-fO8EArNw,T,None,NaN,NaN,b-ZyMZo4Bew,None,NaN
2321180,UgyPEP4wTHbtRX6Cyhx4AaABAg,UCJN_T5pSklCrr8WTmd8TLXQ,Mekk,None,NaN,NaN,b-ZyMZo4Bew,None,NaN
2321181,Ugwr5Jg6TWP1CBvcP7F4AaABAg,UCK2LGyG60cPymaOVL2sbuXQ,Sir William and,None,NaN,NaN,b-ZyMZo4Bew,None,NaN


In [11]:
now = datetime.now()
filename = f'youtube_{now}.csv'
df.to_csv(f'./output/{filename}', index = False)

In [13]:
s3 = utils.S3_Manager()
s3.upload_to_bucket(filename=f'./output/{filename}', dirname='DataCollection')

Done!
